In [1]:
import pandas as pd
man_df = pd.read_csv('data/caesar_all_fr.csv', encoding='utf-8', sep=';')
man_df=man_df.sample(frac=1)
caesar_female = man_df[man_df['sex'] == 0]
caesar_female=caesar_female.loc[:, (caesar_female != 0).any()]

In [2]:
# Drop rows which include NaN
caesar_female = caesar_female.dropna()

In [3]:
caesar_female.isna().sum()

taille                                                0
age                                                   0
weight                                                0
tour_de_cheville                                      0
longueur_du_bras                                      0
tour_de_poitrine                                      0
hauteur_de_poitrine                                   0
hauteur_d_entrejambe                                  0
hauteur_des_hanches                                   0
hauteur_des_genoux                                    0
tour_du_cou                                           0
largeur_d_epaule                                      0
hauteur_des_epaules                                   0
largeur_des_epaules_a_l_horizontales                  0
longueur_de_la_colonne_vertebrale_jusqu_au_poignet    0
tour_de_cuisse                                        0
tour_de_taille                                        0
hauteur_de_la_taille                            

In [4]:
caesar_female["tour_de_hanches_ratio"] = caesar_female["tour_de_hanches"] / caesar_female["taille"]
caesar_female["tour_de_taille_ratio"] = caesar_female["tour_de_taille"] / caesar_female["taille"]

In [6]:
def classifier_morphologie(row):
    # Extraction des mesures
    #tour_ventre = row['tour_de_ventre_maximal']
    forme_ventre = row['tour_de_taille_ratio']
    forme_bassin = row['tour_de_hanches_ratio']
    
    
    # --- Catégorie VENTRE (basée sur le ratio ventre/taille) ---
    if forme_ventre < 0.431483:
        ventre = "plat"
    elif 0.431483 <= forme_ventre<0.520135:
        ventre = "moyen"
    else:
        ventre = "rond"
    
    # --- Catégorie BASSIN (basée sur le tour de hancehs ) ---
    if forme_bassin < 0.588740:
        bassin = "etroit"
    elif 0.588740 <=forme_bassin <0.662996 :
        bassin = "moyen"
    else:
        bassin = "large"
    

    
    return pd.Series([ventre, bassin], 
                     index=['categorie_ventre', 'categorie_bassin'])
    
# Remplace `df` par le nom de ton DataFrame
caesar_female[['categorie_ventre', 'categorie_bassin']] = caesar_female.apply(classifier_morphologie, axis=1)

In [8]:
caesar_female["bonnet"]=caesar_female["tour_de_poitrine"]-caesar_female["tour_de_sous_poitrine"]

In [9]:
def calculer_bonnet(diff):
    if pd.isna(diff):
        return None
    elif diff < 13:
        return 'AA'
    elif diff < 15:
        return 'A'
    elif diff < 17:
        return 'B'
    elif diff < 19:
        return 'C'
    elif diff < 21:
        return 'D'
    elif diff < 23:
        return 'E'
    elif diff < 25:
        return 'F'
    elif diff < 27:
        return 'G'
    elif diff < 29:
        return 'H'
    elif diff < 31:
        return 'I'
    elif diff < 33:
        return 'J'
    else:
        return 'K'

caesar_female['bonnet_lettre'] = caesar_female['bonnet'].map(calculer_bonnet)

def arrondir_taille(val):
    if pd.isna(val):
        return None
    return int(5 * round(val / 5))

caesar_female['taille_soutien_gorge'] = caesar_female['taille_de_poitrine'].apply(arrondir_taille)

caesar_female.head(10)

,taille,age,weight,tour_de_cheville,longueur_du_bras,tour_de_poitrine,hauteur_de_poitrine,hauteur_d_entrejambe,hauteur_des_hanches,hauteur_des_genoux,...,taille_de_poitrine,bonnet_de_soutien_gorge,tour_de_hanches,tour_de_hanches_ratio,tour_de_taille_ratio,categorie_ventre,categorie_bassin,bonnet,bonnet_lettre,taille_soutien_gorge
1789,170.9928,20.0,66.002172,25.1968,60.2996,94.9960,123.4694,81.7118,84.9122,46.8122,...,86.36,1.0,101.1936,0.591800,0.495989,moyen,moyen,14.8844,A,85
33,179.1970,43.0,78.471416,24.6126,66.1924,98.5012,129.9972,85.5980,91.1098,50.6984,...,91.44,3.0,109.7026,0.612190,0.464918,moyen,moyen,19.3040,D,90
889,170.5102,20.0,75.001437,28.3972,59.3090,96.3930,128.5240,82.8040,85.3948,47.2059,...,86.36,1.0,105.5878,0.619246,0.466855,moyen,moyen,10.0838,AA,85
2642,153.5938,36.0,47.173568,20.1930,57.4040,79.9084,112.8014,70.7898,76.0984,42.5958,...,86.36,2.0,92.5068,0.602282,0.446006,moyen,moyen,8.4074,AA,85
2336,161.1122,49.0,49.668324,23.0124,58.1914,81.2038,115.0112,71.0946,76.6064,43.5102,...,86.36,2.0,94.1070,0.584108,0.382942,plat,etroit,12.7000,AA,85
1650,161.1884,57.0,65.698265,23.0886,56.0070,97.5106,110.6170,75.4126,74.0918,42.8625,...,86.36,4.0,103.3018,0.640876,0.481406,moyen,moyen,19.1008,D,85
279,173.6090,49.0,72.347924,25.0952,62.6110,89.5096,124.9934,82.8040,83.3120,49.5046,...,91.44,1.0,107.4928,0.619166,0.417557,plat,moyen,9.4996,AA,90
814,159.1056,49.0,80.399182,25.6032,56.8960,110.2106,110.4646,72.2122,73.1012,41.9735,...,101.60,3.0,112.5982,0.707695,0.588921,rond,large,19.7104,D,100
242,160.6042,55.0,76.498291,25.7048,57.5056,101.4984,114.4016,75.4126,75.4888,43.6372,...,96.52,1.0,107.7976,0.671200,0.612684,rond,large,13.4874,A,95
2649,167.6908,25.0,57.379388,24.2062,56.9976,84.7090,118.5926,76.3016,78.3082,45.5930,...,91.44,2.0,97.5106,0.581490,0.412602,plat,etroit,10.6172,AA,90


In [10]:
# Dictionnaire de correspondance bonnet → rang
bonnet_scale = {
    'AA': 0, 'A': 1, 'B': 2, 'C': 3, 'D': 4,
    'E': 5, 'F': 6, 'G': 7, 'H': 8, 'I': 9, 'J': 10, 'K': 11
}

# Appliquer les rangs
caesar_female['bonnet_rang'] = caesar_female['bonnet_lettre'].map(bonnet_scale)
caesar_female.head(10)

,taille,age,weight,tour_de_cheville,longueur_du_bras,tour_de_poitrine,hauteur_de_poitrine,hauteur_d_entrejambe,hauteur_des_hanches,hauteur_des_genoux,...,bonnet_de_soutien_gorge,tour_de_hanches,tour_de_hanches_ratio,tour_de_taille_ratio,categorie_ventre,categorie_bassin,bonnet,bonnet_lettre,taille_soutien_gorge,bonnet_rang
1789,170.9928,20.0,66.002172,25.1968,60.2996,94.9960,123.4694,81.7118,84.9122,46.8122,...,1.0,101.1936,0.591800,0.495989,moyen,moyen,14.8844,A,85,1
33,179.1970,43.0,78.471416,24.6126,66.1924,98.5012,129.9972,85.5980,91.1098,50.6984,...,3.0,109.7026,0.612190,0.464918,moyen,moyen,19.3040,D,90,4
889,170.5102,20.0,75.001437,28.3972,59.3090,96.3930,128.5240,82.8040,85.3948,47.2059,...,1.0,105.5878,0.619246,0.466855,moyen,moyen,10.0838,AA,85,0
2642,153.5938,36.0,47.173568,20.1930,57.4040,79.9084,112.8014,70.7898,76.0984,42.5958,...,2.0,92.5068,0.602282,0.446006,moyen,moyen,8.4074,AA,85,0
2336,161.1122,49.0,49.668324,23.0124,58.1914,81.2038,115.0112,71.0946,76.6064,43.5102,...,2.0,94.1070,0.584108,0.382942,plat,etroit,12.7000,AA,85,0
1650,161.1884,57.0,65.698265,23.0886,56.0070,97.5106,110.6170,75.4126,74.0918,42.8625,...,4.0,103.3018,0.640876,0.481406,moyen,moyen,19.1008,D,85,4
279,173.6090,49.0,72.347924,25.0952,62.6110,89.5096,124.9934,82.8040,83.3120,49.5046,...,1.0,107.4928,0.619166,0.417557,plat,moyen,9.4996,AA,90,0
814,159.1056,49.0,80.399182,25.6032,56.8960,110.2106,110.4646,72.2122,73.1012,41.9735,...,3.0,112.5982,0.707695,0.588921,rond,large,19.7104,D,100,4
242,160.6042,55.0,76.498291,25.7048,57.5056,101.4984,114.4016,75.4126,75.4888,43.6372,...,1.0,107.7976,0.671200,0.612684,rond,large,13.4874,A,95,1
2649,167.6908,25.0,57.379388,24.2062,56.9976,84.7090,118.5926,76.3016,78.3082,45.5930,...,2.0,97.5106,0.581490,0.412602,plat,etroit,10.6172,AA,90,0


In [11]:
# 1. Wrapper
class MultiTPOTWrapper:
    def __init__(self, models):
        self.models = models

    def fit(self, X, y=None):
        return self

    def predict(self, X):
        import numpy as np
        predictions = [m.predict(X) for m in self.models]
        return np.vstack(predictions).T

In [13]:
# 2. Pré-traitement
from sklearn.preprocessing import RobustScaler, MinMaxScaler,OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
import joblib


X_train=caesar_female[['taille', 'age', 'weight','taille_soutien_gorge',
                       'bonnet_rang','categorie_ventre', 'categorie_bassin']]

# Colonnes numériques et catégorielles
cols_num = ["taille","age","weight","taille_soutien_gorge"]
col_bon=["bonnet_rang"]
cols_cat = ["categorie_ventre", "categorie_bassin"]

# Pipeline pour les colonnes numériques : imputation par médiane + scaling
pipeline_num = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", RobustScaler())
])

pipeline_bon = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", MinMaxScaler())
])

# Pipeline pour les colonnes catégorielles : imputation par la valeur la plus fréquente + one-hot
pipeline_cat = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("encoder", OneHotEncoder(handle_unknown="ignore"))
])

# Pipeline globale de prétraitement
preprocessor = ColumnTransformer(transformers=[
    ("num", pipeline_num, cols_num),
    ("bon", pipeline_bon, col_bon),
    ("cat", pipeline_cat, cols_cat)

])

# Fit sur X_train (tu dois définir X_train comme DataFrame brut)
preprocessor.fit(X_train)

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('scaler', RobustScaler())]),
                                 ['taille', 'age', 'weight',
                                  'taille_soutien_gorge']),
                                ('bon',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('scaler', MinMaxScaler())]),
                                 ['bonnet_rang']),
                                ('cat',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('encoder',
                                                  OneHotEncoder(handle_unknown='ignore'))]),
                                 ['categorie_ventre', 'categorie_bassin'])])

In [14]:
import joblib
import os

# 📁 Répertoire contenant les fichiers .pkl
dossier_modeles = "./modeles_female_tpot_complet"  # modifie ce chemin si nécessaire

# 📌 Chargement de tous les modèles
modeles = {}
for nom_fichier in os.listdir(dossier_modeles):
    if nom_fichier.endswith(".pkl"):
        nom_cible = nom_fichier.replace("tpot_", "").replace(".pkl", "")
        chemin_complet = os.path.join(dossier_modeles, nom_fichier)
        modeles[nom_cible] = joblib.load(chemin_complet)

print(f"✅ {len(modeles)} modèles chargés :", list(modeles.keys()))

✅ 20 modèles chargés : ['hauteur_des_epaules', 'hauteur_des_genoux', 'hauteur_des_hanches', 'hauteur_de_la_taille', 'hauteur_de_poitrine', 'hauteur_d_entrejambe', 'largeur_des_epaules_a_l_horizontales', 'largeur_de_mamelon_a_mamelon', 'largeur_d_epaule', 'longueur_de_la_colonne_vertebrale_jusqu_au_poignet', 'longueur_du_bras', 'longueur_d_avant_bras', 'taille_de_poitrine', 'tour_de_cheville', 'tour_de_cuisse', 'tour_de_hanches', 'tour_de_poitrine', 'tour_de_sous_poitrine', 'tour_de_taille', 'tour_du_cou']


In [15]:
import os

dossier_modeles = "./modeles_female_tpot_complet"  # adapte ce chemin
noms_fichiers = [f for f in os.listdir(dossier_modeles) if f.endswith(".pkl")]

# Dossier de sauvegarde des pipelines complets
dossier_output = "pipelines_female_complets"
os.makedirs(dossier_output, exist_ok=True)

for nom_fichier in noms_fichiers:
    chemin_modele = os.path.join(dossier_modeles, nom_fichier)
    
    # Charger le modèle TPOT
    tpot_model = joblib.load(chemin_modele)
    
    # Créer le pipeline complet
    full_pipeline = Pipeline(steps=[
        ("preprocess", preprocessor),
        ("model", tpot_model)
    ])
    
    # Sauvegarder le pipeline complet
    nom_fichier_pipeline = f"pipeline_{nom_fichier}"
    chemin_output = os.path.join(dossier_output, nom_fichier_pipeline)
    joblib.dump(full_pipeline, chemin_output)

    print(f"✔ Pipeline sauvegardé : {chemin_output}")

✔ Pipeline sauvegardé : pipelines_female_complets\pipeline_tpot_hauteur_des_epaules.pkl
✔ Pipeline sauvegardé : pipelines_female_complets\pipeline_tpot_hauteur_des_genoux.pkl
✔ Pipeline sauvegardé : pipelines_female_complets\pipeline_tpot_hauteur_des_hanches.pkl
✔ Pipeline sauvegardé : pipelines_female_complets\pipeline_tpot_hauteur_de_la_taille.pkl
✔ Pipeline sauvegardé : pipelines_female_complets\pipeline_tpot_hauteur_de_poitrine.pkl
✔ Pipeline sauvegardé : pipelines_female_complets\pipeline_tpot_hauteur_d_entrejambe.pkl
✔ Pipeline sauvegardé : pipelines_female_complets\pipeline_tpot_largeur_des_epaules_a_l_horizontales.pkl
✔ Pipeline sauvegardé : pipelines_female_complets\pipeline_tpot_largeur_de_mamelon_a_mamelon.pkl
✔ Pipeline sauvegardé : pipelines_female_complets\pipeline_tpot_largeur_d_epaule.pkl
✔ Pipeline sauvegardé : pipelines_female_complets\pipeline_tpot_longueur_de_la_colonne_vertebrale_jusqu_au_poignet.pkl
✔ Pipeline sauvegardé : pipelines_female_complets\pipeline_tpot_l

In [17]:
# 3. Chargement modèles
import joblib
import os

dossier_pipelines = "pipelines_female_complets"
models = []
for filename in sorted(os.listdir(dossier_pipelines)):
    if filename.endswith(".pkl"):
        path = os.path.join(dossier_pipelines, filename)
        models.append(joblib.load(path))
        print(f"✅ Modèle chargé : {filename}")

✅ Modèle chargé : pipeline_tpot_hauteur_d_entrejambe.pkl
✅ Modèle chargé : pipeline_tpot_hauteur_de_la_taille.pkl
✅ Modèle chargé : pipeline_tpot_hauteur_de_poitrine.pkl
✅ Modèle chargé : pipeline_tpot_hauteur_des_epaules.pkl
✅ Modèle chargé : pipeline_tpot_hauteur_des_genoux.pkl
✅ Modèle chargé : pipeline_tpot_hauteur_des_hanches.pkl
✅ Modèle chargé : pipeline_tpot_largeur_d_epaule.pkl
✅ Modèle chargé : pipeline_tpot_largeur_de_mamelon_a_mamelon.pkl
✅ Modèle chargé : pipeline_tpot_largeur_des_epaules_a_l_horizontales.pkl
✅ Modèle chargé : pipeline_tpot_longueur_d_avant_bras.pkl
✅ Modèle chargé : pipeline_tpot_longueur_de_la_colonne_vertebrale_jusqu_au_poignet.pkl
✅ Modèle chargé : pipeline_tpot_longueur_du_bras.pkl
✅ Modèle chargé : pipeline_tpot_taille_de_poitrine.pkl
✅ Modèle chargé : pipeline_tpot_tour_de_cheville.pkl
✅ Modèle chargé : pipeline_tpot_tour_de_cuisse.pkl
✅ Modèle chargé : pipeline_tpot_tour_de_hanches.pkl
✅ Modèle chargé : pipeline_tpot_tour_de_poitrine.pkl
✅ Modèle c

In [18]:
# 4. Pipeline complet
full_pipeline = Pipeline([
    ("preprocess", preprocessor),
    ("multi_model", MultiTPOTWrapper(models))
])

In [19]:
# 5. Sauvegarde
joblib.dump(full_pipeline, "full_pipeline_female.pkl")

# 6. Prédiction
pipeline_loaded = joblib.load("full_pipeline_female.pkl")

In [20]:
# ✅ Créer un wrapper à partir des pipelines chargés
wrapper = MultiTPOTWrapper(models)
new_data = pd.DataFrame([{
    "taille": 187,
    "age": 33,
    "weight": 80,
    "categorie_ventre": "palt",
    "categorie_bassin": "large",
    "taille_soutien_gorge": 95,
    "bonnet_rang":5
    
}])
# ✅ Prédictions
preds = wrapper.predict(new_data)


c:\Users\mbouke.besse\OneDrive - ESTIA\Documents\code-dataset\.venv\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\mbouke.besse\OneDrive - ESTIA\Documents\code-dataset\.venv\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\mbouke.besse\OneDrive - ESTIA\Documents\code-dataset\.venv\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but RobustScaler was fitted with feature names
  warnings.warn(
c:\Users\mbouke.besse\OneDrive - ESTIA\Documents\code-dataset\.venv\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
c:\Users\mbouke.besse\OneDrive - ESTIA\Documents\code-datas

In [21]:
# Suppose que tu as déjà ton pipeline chargé
# pipeline_loaded est de type Pipeline(steps=[("preprocess", ...), ("multi_model", ...)])

# Accès au préprocesseur uniquement
preprocessor = pipeline_loaded.named_steps["preprocess"]

# Applique le transformateur au new_data brut
X = preprocessor.transform(new_data)

# Si tu veux voir sous forme de DataFrame
import pandas as pd

# Récupère les noms des colonnes après transformation
# Cela fonctionne uniquement si le ColumnTransformer est simple
try:
    feature_names = preprocessor.get_feature_names_out()
    X_df = pd.DataFrame(X, columns=feature_names)
except:
    # fallback si les noms ne sont pas récupérables
    X_df = pd.DataFrame(X)

# Affiche les premières lignes
X_df.head()
X_df.columns

Index(['num__taille', 'num__age', 'num__weight', 'num__taille_soutien_gorge',
       'bon__bonnet_rang', 'cat__categorie_ventre_moyen',
       'cat__categorie_ventre_plat', 'cat__categorie_ventre_rond',
       'cat__categorie_bassin_etroit', 'cat__categorie_bassin_large',
       'cat__categorie_bassin_moyen'],
      dtype='object')

In [22]:
# ✅ Extraire les noms des cibles
noms_variables = [f.replace("pipeline_tpot_", "").replace(".pkl", "") 
                  for f in sorted(os.listdir(dossier_pipelines)) if f.endswith(".pkl")]

# ✅ Associer les valeurs
predictions_dict = dict(zip(noms_variables, preds[0]))

# ✅ Affichage
print("📊 Prédictions :")
for k, v in predictions_dict.items():
    print(f" - {k} : {v:.2f}")


📊 Prédictions :
 - hauteur_d_entrejambe : 89.06
 - hauteur_de_la_taille : 115.19
 - hauteur_de_poitrine : 134.73
 - hauteur_des_epaules : 153.39
 - hauteur_des_genoux : 51.76
 - hauteur_des_hanches : 91.83
 - largeur_d_epaule : 34.24
 - largeur_de_mamelon_a_mamelon : 19.49
 - largeur_des_epaules_a_l_horizontales : 44.68
 - longueur_d_avant_bras : 26.52
 - longueur_de_la_colonne_vertebrale_jusqu_au_poignet : 59.63
 - longueur_du_bras : 64.85
 - taille_de_poitrine : 96.52
 - tour_de_cheville : 26.10
 - tour_de_cuisse : 64.03
 - tour_de_hanches : 111.27
 - tour_de_poitrine : 96.02
 - tour_de_sous_poitrine : 84.65
 - tour_de_taille : 64.08
 - tour_du_cou : 43.51


In [27]:
import pandas as pd

# Liste des noms de colonnes
colonnes = [
    "hauteur_d_entrejambe",
    "hauteur_de_la_taille",
    "hauteur_de_poitrine",
    "hauteur_des_epaules",
    "hauteur_des_genoux",
    "hauteur_des_hanches",
    "largeur_d_epaule",
    "largeur_de_mamelon_a_mamelon",
    "largeur_des_epaules_a_l_horizontales",
    "longueur_d_avant_bras",
    "longueur_de_la_colonne_vertebrale_jusqu_au_poignet",
    "longueur_du_bras",
    "taille_de_poitrine",
    "tour_de_cheville",
    "tour_de_cuisse",
    "tour_de_hanches",
    "tour_de_poitrine",
    "tour_de_sous_poitrine",
    "tour_de_taille",
    "tour_du_cou"
]

# Création d'un DataFrame vide avec seulement les colonnes
df = pd.DataFrame(columns=colonnes)

# Enregistrement dans un fichier CSV
df.to_csv("data/mesures_morphologiques_vides.csv", encoding='utf-8', sep=';', index=False)

print("✅ Fichier CSV vide avec colonnes créé : data/mesures_morphologiques_vides.csv")


✅ Fichier CSV vide avec colonnes créé : data/mesures_morphologiques_vides.csv
